# Build the Database

In [0]:
import re
userName = spark.sql("SELECT CURRENT_USER").collect()[0]['current_user()']
userName0 = userName.split("@")[0]
userName0 = re.sub('[!#$%&\'*+-/=?^`{}|\.]+', '_', userName0)
userName1 = userName.split("@")[1]
userName = f'{userName0}@{userName1}'
dbutils.fs.mkdirs(f"/Users/{userName}/data")
userDir = f"/Users/{userName}/data"
databaseName = f"{userName0}_FinalProject_01"

print('databaseName ' + databaseName)
print('UserDir ' + userDir)

spark.sql(f"DROP DATABASE IF EXISTS {databaseName} CASCADE")
spark.sql(f"CREATE DATABASE {databaseName}")
spark.sql(f"use {databaseName}")

print (f"Database {databaseName} successfully rebuilt.")

In [0]:
%fs ls /mnt/data/2023-kaggle-final/store-sales/

## Initial Bronze Tables

In [0]:
rootPath = "dbfs:/mnt/data/2023-kaggle-final/store-sales/"

for file in dbutils.fs.ls(rootPath):
  tableName = "bronze_" + file.name.replace('.csv', '')
  print (f"processing file {file.name} into table name {tableName}...")

  loadDf = spark.read.option("header", True).option("inferSchema", True).csv(file.path)
  loadDf.write.saveAsTable(tableName) #saves delta table
  
  print(f"Successfully saved delta table {tableName}.")
  print("")


# Add DQ Constraints after table creation

## bronze_holidays_events
* added not Null constraint
* added a CHECK to ensure date ranges are after expected value

In [0]:
%sql
ALTER TABLE bronze_holidays_events ALTER COLUMN `type` SET NOT NULL;

In [0]:
%sql
ALTER TABLE bronze_holidays_events ADD CONSTRAINT dateWithinRange CHECK (`date` >= '2012-03-02')

In [0]:
%sql
DESC HISTORY bronze_holidays_events

## bronze_oil
* date field is NOT NULL and after a range

In [0]:
%sql
ALTER TABLE bronze_oil ADD CONSTRAINT dateWithinRange CHECK (`date` >= '2012-03-02');
ALTER TABLE bronze_oil ALTER COLUMN `date` SET NOT NULL;

In [0]:
%sql
DESC HISTORY bronze_oil

## bronze_store
* store_nbr should be greater than 1, cannot be negative
* type should be a list of A, B , C, D, E

In [0]:
%sql
ALTER TABLE bronze_stores ADD CONSTRAINT validIds CHECK(store_nbr>=1) 

In [0]:
%sql
ALTER TABLE bronze_stores ADD CONSTRAINT validType CHECK(`type` IN ("A","B","C","D","E"))

In [0]:
%sql
DESC HISTORY bronze_stores